<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Environment-preparation" data-toc-modified-id="Environment-preparation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Environment preparation</a></span><ul class="toc-item"><li><span><a href="#Libraries-and-functions" data-toc-modified-id="Libraries-and-functions-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Libraries and functions</a></span></li></ul></li><li><span><a href="#Property-values-and-propensity-to-be-sold" data-toc-modified-id="Property-values-and-propensity-to-be-sold-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Property values and propensity to be sold</a></span><ul class="toc-item"><li><span><a href="#Linear-Regression-method" data-toc-modified-id="Linear-Regression-method-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Linear Regression method</a></span><ul class="toc-item"><li><span><a href="#Price-Distribution" data-toc-modified-id="Price-Distribution-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Price Distribution</a></span></li><li><span><a href="#Bivariate-Analysis---log(price)" data-toc-modified-id="Bivariate-Analysis---log(price)-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Bivariate Analysis - log(price)</a></span></li><li><span><a href="#Linear-Model-log(price)-of-sold-properties" data-toc-modified-id="Linear-Model-log(price)-of-sold-properties-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Linear Model log(price) of sold properties</a></span></li><li><span><a href="#Selling-price-prediction-of-unsold-properties" data-toc-modified-id="Selling-price-prediction-of-unsold-properties-2.1.4"><span class="toc-item-num">2.1.4&nbsp;&nbsp;</span>Selling price prediction of unsold properties</a></span></li></ul></li><li><span><a href="#Logistic-Regression-method" data-toc-modified-id="Logistic-Regression-method-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Logistic Regression method</a></span><ul class="toc-item"><li><span><a href="#Bivariate-Analysis---Sold/Unsold" data-toc-modified-id="Bivariate-Analysis---Sold/Unsold-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Bivariate Analysis - Sold/Unsold</a></span></li><li><span><a href="#Logistic-Model---Propensity-to-be-sold" data-toc-modified-id="Logistic-Model---Propensity-to-be-sold-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Logistic Model - Propensity to be sold</a></span></li><li><span><a href="#Conditional-probabilities" data-toc-modified-id="Conditional-probabilities-2.2.3"><span class="toc-item-num">2.2.3&nbsp;&nbsp;</span>Conditional probabilities</a></span></li><li><span><a href="#Optimall-selling-price-strategy-unsold-properties" data-toc-modified-id="Optimall-selling-price-strategy-unsold-properties-2.2.4"><span class="toc-item-num">2.2.4&nbsp;&nbsp;</span>Optimall selling price strategy unsold properties</a></span></li></ul></li></ul></li></ul></div>

# Environment preparation


In [ ]:
if 'google.colab' in str(get_ipython()):
    !git clone https://github.com/griu/msc_python.git /content/msc_python
    !git -C /content/msc_python pull
    %cd /content/msc_python
    !pip install stargazer
    !pip install pyreadstat

__[Open in Colab](https://colab.research.google.com/github/griu/msc_python/blob/master/Property_Value_Propensity_Sell_Seminar.ipynb)__   *: <span style="color:rgba(255, 99, 71, 0.8)">Padawan! When you login to Colab, prepare the environment by running the following code.</span>*

## Libraries and functions

In [ ]:
# Importing the packages

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests

import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.graphics.api as smg
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
import scipy.stats as stats
from statsmodels.genmod.generalized_linear_model import GLM
from statsmodels.genmod import families
from stargazer.stargazer import Stargazer
from IPython.core.display import display, HTML
import scipy
import math

# setting matplot theme
sns.set_theme(color_codes=True)

# Visualization
def visreg(fit, xvar, by, data, cond=dict()):
    
    # conditions
    allBy = np.sort(data[by].unique())
    
    # initializing dataframes
    df0 = pd.DataFrame(np.sort(data[xvar].copy().unique()),columns=[xvar])
    df0[by] = allBy[0] 
    df1 = df0.copy()
    
    # several copies by
    for x in allBy[1:]:
        df1[by] = x
        df0 = df0.append(df1, ignore_index=True)
        
    for k, v in cond.items():
        df0[k] = v
    
    # predict
    df0[fit.model.endog_names] = fit.predict(df0)
    
    ax = sns.lineplot(data = df0, x=xvar, y = fit.model.endog_names, hue = by)
    ax.set_title('Conditional effects plot')
    ax.set(ylabel='Prob(' + fit.model.endog_names + ')')

    return ax

# model comparision
def anovaLogit(fit1,fit2):
    Deviance = abs(-2*(fit2.model.loglike(fit2.params) - fit1.model.loglike(fit1.params)))
    DF = 2
    p_value = scipy.stats.chi2.sf(Deviance,DF)
    
    return {'Deviance':Deviance, 'DF':DF, 'p_value' : p_value}
    

# defining bivariate functions

def without_hue(ax, feature):
    total = len(feature)
    for p in ax.patches:
        percentage = '{:.1f}%'.format(100 * p.get_height()/total)
        x = p.get_x() + p.get_width() / 10
        y = p.get_y() + p.get_height()
        ax.annotate(percentage, (x, y), size = 12)
    plt.show()

def bivar_bin(x,catVar,binVar,method = 'qcut',points = 7, saveTransf=False):
    # options for method: 'qcut', 'cut', None
    ind_transf = 0
    if saveTransf:
        x1 = x
    else:
        x1 = x.copy()

    # partitioning numerical column
    if (catVar in x1.select_dtypes('number').columns) and (method 
                                                          in ['qcut','cut']):
        if type(points) is list:
            if len(x1[catVar].unique()) > len(points):
                ind_transf = 1
        else:
            if len(x1[catVar].unique()) > points:
                ind_transf = 1
        if (ind_transf==1) and (method=="qcut"):
            x1[catVar + '_T'] = pd.qcut(x1[catVar], points,duplicates='drop')
            catVar = catVar + '_T'

        if (ind_transf==1) and (method=="cut"):
            x1[catVar + '_T'] = pd.cut(x1[catVar], points,right=False,duplicates='drop')
            catVar = catVar + '_T'
    
    # generating plots
    sns.set(rc={'figure.figsize':(11,4)})
    fig, ax = plt.subplots(1,2)
    
    sns.countplot(x=catVar,data=x1, ax=ax[0])
    ax[0].tick_params(labelrotation=90)
    ax[0].set(title=catVar+' distribution')
    
    sns.barplot(x=catVar,y=binVar, data=x1, ax=ax[1])
    ax[1].tick_params(labelrotation=90)
    ax[1].set(ylabel='Propensity '+binVar, title=binVar + ' % by '+catVar)
    
    # adding percentage to countplot
    without_hue(ax[0],x1[catVar])


# Property values and propensity to be sold 

The following data set corresponds to a sample of 1,000 house units in Spain on-sell during the 2008 crises:
`properties.csv`. The variable price is the offer price if variable sold = 0 and the transaction value if sold = 1 .

Your client is a real estate company and your mission is to help them to sell all the unsold properties by maximizing the profit from their sale.

1. *Linear regression method:*
   1. Inspect the distribution of the variable price and comment on whether to use the transformation log(price).
   1. Do a bivariate analysis between log(price) and other relevant explanatory factors in the sample.
   1. Use the log(price) as a dependent variable to fit what you feel is a relevant/useful regression model. Do this selecting only the sample of sold properties. Discuss the relevance of these regressions to understand the determinants of variable price. Summarize the findings that you find useful to inform your client.
   1. Using the model adjusted in the previous step, predict the selling price of the sample of unsold properties. Summarize the unsold properties for each region with the following key performance indicators (KPIs): Nr of properties, total offering price, total selling price, and average deviation (ADEV) defined like: (sum(selling price) / sum(offer price) - 1). comment the table.
1. *Logistic regression method:*
   1. Do a bivariate analysis between sold variable and other relevant explanatory factors in the sample.
   1. Adjust a logistic regression model with overall sample using sold as a dependent variable. Assess which features of the properties can determine the sold or unsold status of the propertie.
   1. Use the previous model to plot the probability to be sold conditioned to the price by habitat size of a property with 150 square meters, non rural and without pool (add other conditions if you need). Interpret the plot for your client.
   1. Taking the conditional plot from the previous example comment how this can be used to decide the optimal price to sell this property. Comment pros and cons of this method, compared to the selling price proposed in the linear regression section.


## Linear Regression method

Data loading:

In [ ]:
df = pd.read_csv("properties.csv")

#casting region to categorical
df.region = df.region.astype("category")

df.shape

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.describe()

### Price Distribution

In [ ]:
# Hint: look at sns.displot() function with log_scale=True parameter


Comment:

### Bivariate Analysis - log(price)

In [ ]:
# Hint 1: use sns.pairplot(), sns.lmplot() and sns.boxplot() functions. 
# Hint 2: If its encessary, inspect any transformation/interaction that could help to linearize the relations
# Hint 3: Take care of possible high correlations between explanatory variables 


### Linear Model log(price) of sold properties

In [ ]:
# Hint 1: To create the DataFrame with sold properties use the condition [df.sold == 1]
# Hint 2: Comment the R2 and interpret the model with mod1.summary() and partial regression plots.
# Hint 3: Inspect the residuals versus prediction, vif and influential values 


Comment:

### Selling price prediction of unsold properties

In [ ]:
# Hint 1: use mod1.predict(df1) predicts mod1 over df1 and return a pandas Series with predictions.
# Hint 2: df1.groupby([var]).agg({'var1':'stat1','var2':'stat2', ...}) summaryze each var category the statistic stat1 over 
#         var1, stat2 over var2... Common statisics are 'count', 'mean' and 'sum'.
# Hint 3: Ratios can be calulated over the new dataframe from the agregated values.


Comment:

## Logistic Regression method


### Bivariate Analysis - Sold/Unsold

In [ ]:
# Hint 1: bivar_bin() function using the parameters method, points, saveTransf


### Logistic Model - Propensity to be sold

In [ ]:
# Hint 1: Adjust the model with mod2 = sm.Logit.from_formula().fit()
# Hint 2: Interpretation should be done with mod2.summary() and mod2.get_margeff(dummy=True).summary() 
# Hint 3: Influential values could be analyzed with mod3=GLM.from_formula().fit(), mod3.get_influence().plot_influence() and mod3.get_influence().cooks_distance[0]  


Comment:

### Conditional probabilities

In [ ]:
# use visreg(). All explanatory variables, except xvar and by, should have been fixed in the cond dictionary.


Comment:

### Optimall selling price strategy unsold properties

> Hint: Fix the selling price from a global policy of Propensity to be sold. 

Comment: